In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
import register_ as reg
import algo_cooling as algo
import quf
import time
import numpy as np
from tqdm import tqdm
import nlopt
import torch
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from quimb.tensor.belief_propagation.l2bp import L2BP
from gate_arb import TensorNetworkGenVector


import matplotlib as mpl
import matplotlib.pyplot as plt


In [2]:
reg.reg_complex_svd()

to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)
to_backend_ = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=True)

opt = algo.opt_(progbar=False, max_time="rate:1e9", max_repeats=128, optlib="cmaes")


In [3]:
info_su = qu.load_from_disk(f"store_state/info_su")
# info_bp = qu.load_from_disk(f"store_state/info_bp")

In [4]:
#ITF params
J, h, chi, dt, depth = info_su["J"], info_su["h"], info_su["chi"], info_su["dt"], info_su["depth"],
Lx, Ly, L = info_su["Lx"], info_su["Ly"], info_su["L"]
t = depth * dt
print("t", t, "chi", chi, "L", L, "depth", depth)

t 1.0 chi 20 L 16 depth 20


In [5]:
edges_1d, sites, site_tags = info_su["edges_1d"], info_su["sites"], info_su["site_tags"]

In [6]:
pepo, gauges = info_su["pepo"], info_su["gauges"]
# pepo.apply_to_arrays(to_backend_c)
# to_backend_c(pepo.exponent)
# gauges = { u:to_backend(v)  for u, v in gauges.items()}




In [7]:
# gauges = {}
# pepo = info_bp["pepo"]

# pepo.retag_(info_su["map_tags"])
# pepo.reindex_(info_su["map_inds"])
# pepo.reindex_(info_su["map_inds_b"])

# pepo.view_as_(TensorNetworkGenVector,  site_tag_id='I{}',site_ind_id='k{}',)

# pepo.apply_to_arrays(to_backend_c)

# # pepo.gauge_all_simple_(max_iterations=100, tol=1e-6, gauges=gauges, progbar=True)


In [8]:
pepo.exponent

tensor(2.4082, dtype=torch.float64)

In [9]:
# depth_total = 3
# params = {}

# p1 = 1.0 / (2.0 - 2.0**(1/3))
# p2 = - (2.0**(1/3)) / (2.0 - 2.0**(1/3))


# for depth in range(depth_total):
#         if depth ==1:
#             phi = -h * 0.5 * p2
#             theta = -J * 2 * 0.5 *p2
#         else:
#             phi = -h * 0.5 * p1
#             theta = -J * 2 * 0.5 *p1

#         params[f"rx_depth{depth}"] = to_backend_( torch.tensor( phi ).clone().detach() )
#         params[f"rzz_depth{depth}"] = to_backend_( torch.tensor( theta ).clone().detach() )


depth_total = 5
params = {}
for depth in range(depth_total):
        phi = -0.
        theta = -0.
        params[f"rx_depth{depth}"] = to_backend_( torch.tensor( phi ).clone().detach() )
        params[f"rzz_depth{depth}"] = to_backend_( torch.tensor( theta ).clone().detach() )


/Users/rezah/Documents/time_compressed_algo/algo_cooling.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=dtype, device=device, requires_grad=requires_grad)


In [10]:
%%time
chi = 20
chi_mps = int(chi*1.2)
cost_opts = { "Lx":Lx, "Ly":Ly, "depth_total":depth_total, "opt":opt, "edges":edges_1d, "stable":True  }
cost_opts |= { "cutoff":0.0, "map_tags_2d":info_su["map_tags_2d"], "opt":opt, "chi":chi, "sites": sites }
cost_opts |= { "renorm":False, "equalize_norms":False, "chi_mps":chi_mps }
cost_opts |= { "pepo":pepo, "gauges":gauges  }

cost = algo.cost_function_su(params,  **cost_opts)
print( float(cost) )

0.9999894720899822
CPU times: user 5.1 s, sys: 6.44 s, total: 11.5 s
Wall time: 5.16 s


<timed exec>:9: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:837.)


In [11]:
import torch.optim as optim

params_list = list(params.values())
optimizer = optim.Adam(params_list, lr=0.01)

its_max = 1
loss_history = []
pbar = tqdm(total=its_max, desc="adam", ncols=100, dynamic_ncols=True)

for step in range(its_max):
    optimizer.zero_grad()

    with torch.no_grad():
        pepo_ = pepo.copy()
        gauges_ = {u: v * 1.0 for u, v in gauges.items()}
        cost_opts |= { "pepo":pepo_, "gauges":gauges_  }
    

    loss = algo.cost_function_su(params,  **cost_opts)
    loss.backward()  # keep graph if needed
    optimizer.step()
    
    loss_history.append(loss.item())   # store scalar for plotting
    
    print(f"Step {step}: Loss = {loss.item()}")
    pbar.set_postfix({
            "loss": f"{loss.item():.3e}",
        })
    pbar.update(1)
pbar.close()


adam: 100%|██████████| 1/1 [00:11<00:00, 11.15s/it, loss=1.000e+00]

Step 0: Loss = 0.9999894720899822


In [12]:
cost_opts["chi"] = 20
params, energy_hist, grad_hist = algo.nlopt_optimize_(params, cost_opts, device="cpu", optimizer="lbfgs",
                                                     cost_fn=algo.cost_function_su, its_max=40
                                                    )

nlopt:  45%|████▌     | 18/40 [03:03<03:37,  9.90s/it, loss=9.989e-01, ||grad||=2.13e-02]

runtime_error: 

In [ ]:
params

In [ ]:
cost_opts["chi"] = 16
cost = algo.cost_function_su(params,  **cost_opts)
print( float(cost) )

In [ ]:
import torch
import matplotlib.pyplot as plt


# Convert to CPU and detach before plotting
names = list(params.keys())
values = [v.detach().cpu().item() for v in params.values()]

plt.figure(figsize=(8, 4))
plt.plot(names, values, marker='o', linestyle='-', color='blue')
plt.xticks(rotation=45)
plt.ylabel("Parameter value")
plt.title("RX and RZZ parameter values by depth")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))  # compact and clean figure



plt.plot(
    loss_history, zorder=5, label=r"$\chi=16$",color="orange",
    markersize=12, linewidth=2.2, alpha=1., marker='*', linestyle='-',
    markeredgecolor="whitesmoke", mfc="orange", markeredgewidth=1.4
)

plt.yscale('log')

# Set axis limits
# plt.xlim(0, max(t_l))        # adjust x-axis range if needed
# plt.ylim(0.99, 0.99999)            # adjust y-axis range for log scale

plt.ylabel(r"$ 1- \mathrm{Tr}\!\left(U^\dagger V(\theta)\right) / 2^L$", fontsize=12)
plt.xlabel("Time", fontsize=12)

plt.grid(True, linestyle='--', linewidth=0.3, alpha=0.5)
plt.tick_params(width=0.8, labelsize=12)
for spine in plt.gca().spines.values():
    spine.set_linewidth(1.)


plt.title(rf"Ising Model, $J=1, h=3.05$, $L = {4} \times {4}$, $\delta={0.05}$", fontsize=14, color='darkred', pad=15)  # pad adds space above the plot

plt.grid(color='gray', linestyle='-', linewidth=1., alpha=1.)
plt.legend(loc='best', frameon=True, shadow=True, fontsize=14)
plt.tight_layout()
plt.show()
